In [51]:
# Web scraping libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

# Other necessary libraries
from datetime import datetime
import time
import os
import requests
import warnings 

warnings.filterwarnings('ignore')  # Ignore warnings
start = datetime.now()  # Start measuring execution time

ua = UserAgent()
user_agent = ua.random
headers = {'user-agent': user_agent}

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver', options=options)
driver.maximize_window()
driver.implicitly_wait(5)

# Initial settings for the website to be scraped
url = 'https://tenjo.tw/category/taipei-eat/page/'
count = len(os.listdir('./')) - 2  # Number of articles already scraped in the folder
page_num = count // 10  # Each page contains ten articles
page = page_num + 1  # Current page number
start_page = page
print('This is the page: ', page)

url = f'{url}{page}'  # Set the URL for the current page
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

title = soup.find_all('h2', {'class': 'entry-title'})  # Find titles of each article

total_page_num = 0  # Total number of articles scraped
article = 0  # Current article index on the page

stop_words = ['*', ':', '<', '>', '?', '|']  # Special characters not allowed in file paths

time.sleep(2)
while True:
    page_title = title[article].text  # Get the title of the current article
    page_title_no_space = ' '.join(page_title.split('&nbsp'))  # Remove '&nbsp' from the title if present

    path = f'./{page_title_no_space}'
    try:
        if not os.path.isdir(path):  # Create a folder for the article if it doesn't exist
            os.makedirs(path)  # Folder name is the article title
    except:
        for i in stop_words:
            path = path.replace(i, '')  # Remove special characters
        if not os.path.isdir(path):  # Create a folder for the article if it doesn't exist
            os.makedirs(path)  # Folder name is the article title

    content = driver.find_element(By.LINK_TEXT, page_title_no_space)
    content.click()  # Open the current article

    print('start to crawl: ', page_title_no_space)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    image = soup.find_all('div', {'class': 'entry-content'})
    filter_image = image[0].find_all('img', {'decoding': 'async'})  # Find all images in the content

    # Download and save images to the folder
    for j in filter_image:
        picture = j.get('src')
        response = requests.get(picture)
        with open(f'{path}/{os.path.basename(picture)}', 'wb') as file:
            file.write(response.content)
    print(page_title_no_space, ' is done!')

    driver.back()
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(2)

    article += 1  # Move to the next article on the page
    total_page_num += 1
    
    # If all articles on the current page are scraped
    if article == len(title):
        page += 1
        try:
            # Navigate to the previous page
            previous_page = driver.find_element(By.LINK_TEXT, '較舊的文章')
            previous_page.click()
            print('This is the page: ', page)

            soup = BeautifulSoup(driver.page_source, 'html.parser')
            title = soup.find_all('h2', {'class': 'entry-title'})  # Find titles of each article

            article = 0  # Reset article index for the new page
            time.sleep(2)
        except:
            # Last page reached
            print('This is the final page!')
            count = len(os.listdir('./')) - 2  # Number of articles already scraped
            print('Total articles: ', count)
            break
    
    # Decide how many pages to crawl
    crawled_page = 60
    if page - start_page == crawled_page:
        print(f'{crawled_page} pages are crawled!')
        count = len(os.listdir('./')) - 2  # Number of articles already scraped
        print('Total crawled articles now: ', count)
        break

end = datetime.now()
print('Total processing time: ', end - start)  # Record total execution time


Could not find local data/json file or could not parse the contents. Fallback to external resource.
Traceback (most recent call last):
  File "c:\Users\passerbyK\anaconda3\lib\site-packages\fake_useragent\utils.py", line 120, in load
    ilr.files("fake_useragent.data").joinpath("browsers.json").read_text()
  File "c:\Users\passerbyK\anaconda3\lib\importlib\resources.py", line 147, in files
    return _common.from_package(_get_package(package))
  File "c:\Users\passerbyK\anaconda3\lib\importlib\_common.py", line 14, in from_package
    return fallback_resources(package.__spec__)
  File "c:\Users\passerbyK\anaconda3\lib\importlib\_common.py", line 18, in fallback_resources
    package_directory = pathlib.Path(spec.origin).parent
  File "c:\Users\passerbyK\anaconda3\lib\pathlib.py", line 1082, in __new__
    self = cls._from_parts(args, init=False)
  File "c:\Users\passerbyK\anaconda3\lib\pathlib.py", line 707, in _from_parts
    drv, root, parts = self._parse_args(args)
  File "c:\Users

This is the page:  170
start to crawl:  【台北食記】嗆頂級麻辣鴛鴦火鍋 平價吃到飽好選擇
【台北食記】嗆頂級麻辣鴛鴦火鍋 平價吃到飽好選擇  is done!
start to crawl:  【台北食記】柊浅川火鍋專門店 活跳跳海鮮與香濃牛奶鍋的完美搭配 東區信義區好吃火鍋推薦
【台北食記】柊浅川火鍋專門店 活跳跳海鮮與香濃牛奶鍋的完美搭配 東區信義區好吃火鍋推薦  is done!
start to crawl:  【台北食記】亞坤咖啡店 康熙來了小S推薦 咖椰吐司新加坡傳統美食 內湖早餐下午茶推薦
【台北食記】亞坤咖啡店 康熙來了小S推薦 咖椰吐司新加坡傳統美食 內湖早餐下午茶推薦  is done!
start to crawl:  【台北食記】若葉鯛魚燒 堅持日本傳統職人技術 單剪鯛魚燒 酥脆薄皮內餡多 中山區遼寧夜市必吃美食
【台北食記】若葉鯛魚燒 堅持日本傳統職人技術 單剪鯛魚燒 酥脆薄皮內餡多 中山區遼寧夜市必吃美食  is done!
start to crawl:  【台北食記】時尚歐式餐廳 大份量早午餐brunch綜合肉吃超飽 TJB CAFÉ 東區國父紀念館美食
【台北食記】時尚歐式餐廳 大份量早午餐brunch綜合肉吃超飽 TJB CAFÉ 東區國父紀念館美食  is done!
start to crawl:  【台北食記】和歌壽司 超澎湃握壽司 海膽干貝天使紅蝦一次滿足 台北車站西門町日本料理推薦
【台北食記】和歌壽司 超澎湃握壽司 海膽干貝天使紅蝦一次滿足 台北車站西門町日本料理推薦  is done!
start to crawl:  【台北食記】超牽絲四起司野菜pizza！搭配蜂蜜更是好吃　Potager Garden菠啾花園 蔬食素材餐廳推薦 *已結束營業
【台北食記】超牽絲四起司野菜pizza！搭配蜂蜜更是好吃　Potager Garden菠啾花園 蔬食素材餐廳推薦 *已結束營業  is done!
start to crawl:  【台北食記】印度風咖哩茶坊 選擇超多的百元餐館 信義區永吉路30巷
【台北食記】印度風咖哩茶坊 選擇超多的百元餐館 信義區永吉路30巷  is done!
start to crawl:  【台北食記】椰篱私房料理鍋物 胡椒豬肚雞 主廚無菜單私房菜 大安區
